In [9]:
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import fft, signal, stats
from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from collections import defaultdict as dd
import os
import sklearn

plt.style.use('seaborn-whitegrid')

In [10]:
dfs = []
for dirname, _, filenames in os.walk('./data'):
    for filename in filenames:
        
        # the gearbox state is in the filename prefix (h = healthy, b = broken tooth)
        state = filename[0]
        
        # the load is in the filename suffix
        load = int(filename.split('.')[0][5:])
        
        # read in the file
        df = pd.read_csv(os.path.join(dirname, filename))
        
        # include the healthy/broken state
        df['state'] = state
        
        # include the load
        df['load'] = load  
        # append to a list
        dfs.append(df)


# concatenate all the datasets and reset the index
df = pd.concat(dfs).reset_index().rename(columns={'index':'sample_index'})

In [11]:
import json
example = df[df["load"] == 50].head(15).to_dict()

with open("example.json", "w") as f:
    json.dump(example, f)




In [12]:
sensor_readings = df.melt(
    id_vars=['sample_index','state','load'],
    value_vars=['a1','a2','a3','a4'],
    var_name='sensor',
    value_name='reading'
)

In [13]:
data = []
labels = []
for (state,load,sensor),g in sensor_readings.groupby(['state','load','sensor']):
    vals = g.reading.values
    splits = np.split(vals, range(1000,vals.shape[0],1000))
    for s in splits[:-1]:  # except the last one
        data.append({
            'sensor_a1': int(sensor=='a1'),
            'sensor_a2': int(sensor=='a2'),
            'sensor_a3': int(sensor=='a3'),
            # no need to put a4: if a1-3 are 0, then it's sensor a4
            'load': load,
            'mean': np.mean(s),
            'std': np.std(s),
            'kurt': stats.kurtosis(s),
            'skew': stats.skew(s),
            'moment': stats.moment(s),
        })
        labels.append(int(state=='b'))  # positive label indicates fault

df_data = pd.DataFrame(data)
# print(df_data)
data = df_data.values
labels = np.array(labels)
(data.shape, labels.shape)

((8036, 9), (8036,))

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    data, labels,
    train_size=0.8,
    random_state=42,
    stratify=labels
)
print(f'Training data: {X_train.shape}')
print(f'Test data: {X_test.shape}')

Training data: (6428, 9)
Test data: (1608, 9)


In [15]:
X_train

array([[ 0.        ,  0.        ,  1.        , ...,  1.40566648,
        -0.18787389,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.97203532,
        -0.16960444,  0.        ],
       [ 0.        ,  1.        ,  0.        , ...,  1.4923866 ,
        -0.43313601,  0.        ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  2.01183795,
        -0.29975252,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  1.3796897 ,
        -0.12181125,  0.        ],
       [ 1.        ,  0.        ,  0.        , ...,  3.17744438,
        -0.10359447,  0.        ]])

In [16]:
X_test

array([[ 1.        ,  0.        ,  0.        , ...,  0.39731981,
         0.246841  ,  0.        ],
       [ 1.        ,  0.        ,  0.        , ...,  1.056462  ,
        -0.10219856,  0.        ],
       [ 1.        ,  0.        ,  0.        , ...,  5.96614868,
        -0.26339802,  0.        ],
       ...,
       [ 0.        ,  0.        ,  1.        , ...,  0.84370205,
        -0.12008536,  0.        ],
       [ 1.        ,  0.        ,  0.        , ...,  2.13733522,
        -0.04298808,  0.        ],
       [ 0.        ,  1.        ,  0.        , ...,  1.4761229 ,
        -0.20018693,  0.        ]])

In [18]:
import numpy as np
import pandas as pd
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split

exported_pipeline = ExtraTreesClassifier(bootstrap=True, criterion="entropy", max_features=0.6000000000000001, min_samples_leaf=13, min_samples_split=11, n_estimators=100)
# Fix random state in exported estimator
if hasattr(exported_pipeline, 'random_state'):
    setattr(exported_pipeline, 'random_state', 42)

exported_pipeline.fit(X_train, y_train)
results = exported_pipeline.predict(X_test)

In [23]:
X_test.shape

(1608, 9)

In [22]:
results.shape

(1608,)

In [8]:
import pickle
filename = 'finalized_model.sav'
pickle.dump(exported_pipeline, open(filename, 'wb'))

In [9]:
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)
acc = sklearn.metrics.accuracy_score(np.array(y_test), 
                                     np.array(results))
acc

0.8638059701492538


0.8638059701492538